## Analysis of portfolio vs SPY

In [1]:
import numpy as np
import os
import pandas as pd
# from mpl_finance import candlestick_ohlc
import pandas_datareader.data as pdr
import datetime
import sys
import re
import sel_scrape as sela

if os.path.abspath('.')  not in sys.path:
    if os.path.abspath('.') not in sys.path:
        sys.path.append(os.path.abspath('.'))
if os.path.abspath('..')  not in sys.path:
    if os.path.abspath('..') not in sys.path:
        sys.path.append(os.path.abspath('..'))

from tqdm import tqdm
%matplotlib inline
import plot_utilities as pu
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pathlib
SYSTEM_HOME = pathlib.Path.home()


Barchart OnDemand Client: https://marketdata.websol.barchart.com/
Barchart OnDemand Client: https://marketdata.websol.barchart.com/


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import importlib
# importlib.reload(pu)

### Read portfolio

In [25]:
# PORTFOLIO_PATH = f'{SYSTEM_HOME}/test_portfolio_jrh.csv'
PORTFOLIO_PATH = f'{SYSTEM_HOME}/test_portfolio_spy_vs_mutual_fund.csv'
df_port = pd.read_csv(PORTFOLIO_PATH)

In [26]:
df_port

,symbol,position
0,XGPMX,38


### Get Market Data for the portfolio


In [27]:
YEARS = 5
df_all = None
for i in range(len(df_port)):
    symbol = df_port.iloc[i].symbol
    shares = df_port.iloc[i].position
    df_temp = pu.get_yahoo(symbol,days_to_fetch=365*YEARS)
    df_temp = df_temp[['trade_date','close']].rename(columns={'close':symbol})
    df_temp[symbol] = df_temp[symbol]*shares
    if df_all is None:
        df_all = df_temp.copy()
    else:
        df_all = df_all.merge(df_temp,on='trade_date',how='inner')
        


get_history: XGPMX FETCHING DATA 2019-11-20 08:52:57.474811


### Do prelim analysis, calculating sharpe ratio of who time period

In [28]:
df_all2 = df_all.copy()
cols = [c for c in df_all2.columns.values if 'trade_' not in c]
df_all2['tot'] = df_all2.apply(lambda r:sum([r[c] for c in cols]),axis=1)
df_all2['port_pctchg'] = df_all2.tot.pct_change()
m = df_all2.port_pctchg.mean()
s = df_all2.port_pctchg.std()
r = m/s * 256**.5
{'daily_mean_return':m,'daily_std':s,'yearl_mean_return':m*256,'yearly_std':s*256**.5,'yearly_sharpe_ratio':r}

{'daily_mean_return': 8.82842429187189e-05,
 'daily_std': 0.00987104316370216,
 'yearl_mean_return': 0.022600766187192037,
 'yearly_std': 0.15793669061923457,
 'yearly_sharpe_ratio': 0.14310016310066692}

### Rolling annualized return, std and  sharpe

In [29]:
ROLLING_PERIOD = 100 # business days
df_all3 = df_all2[['trade_date','tot','port_pctchg']].copy()
df_all3['rolling_annualized_std'] = df_all3.port_pctchg.rolling(ROLLING_PERIOD).std() * 256**.5
df_all3['rolling_annualized_mean'] = df_all3.port_pctchg.rolling(ROLLING_PERIOD).mean() * 256
df_all3['rolling_annualized_sharpe'] = df_all3.rolling_annualized_mean / df_all3.rolling_annualized_std
df_all4 = df_all3[['trade_date','tot','rolling_annualized_mean','rolling_annualized_std','rolling_annualized_sharpe']]
df_all4.tail(100)


,trade_date,tot,rolling_annualized_mean,rolling_annualized_std,rolling_annualized_sharpe
1157,20190701,301.720002,0.219652,0.122204,1.797419
1158,20190702,301.339993,0.236196,0.121466,1.944543
1159,20190703,304.000000,0.252153,0.122096,2.065209
1160,20190705,304.380009,0.248729,0.122069,2.037614
1161,20190708,302.859992,0.202913,0.120898,1.678376
1162,20190709,302.859992,0.193130,0.120805,1.598683
1163,20190710,303.240001,0.202839,0.120691,1.680654
1164,20190711,303.619991,0.179991,0.119737,1.503222
1165,20190712,303.619991,0.170318,0.119638,1.423611
1166,20190715,305.900007,0.183119,0.120104,1.524665


### Plot time-slices of rolling shape ratios using bar graph


In [30]:
VIEW_SLICE = 100
# for i in range(int(len(df_all4)/VIEW_SLICE)):
#     b = i*VIEW_SLICE
#     e = min((i+1)*VIEW_SLICE,len(df_all4)-1)
#     df_view = df_all4.iloc[b:e][['trade_date','rolling_annualized_sharpe']]
#     iplot(pu.plotly_pandas(df_view,x_column='trade_date',bar_plot=True))

### Plot portfolio value and rolling sharpe ratio over whole period

In [31]:
df_to_plot = df_all4[['trade_date','tot','rolling_annualized_sharpe']].copy()
t = 'portfolio value vs sharpe ratio'
p = pu.plotly_pandas(df_to_plot,x_column='trade_date',use_secondary_yaxis=True,plot_title=t)
iplot(p)

###  Do analysis of SPY

In [32]:
df_spy = pu.get_yahoo('SPY',days_to_fetch=365*YEARS)
df_spy2 = df_spy[['trade_date','close']].rename(columns={'close':'spy'})

get_history: SPY FETCHING DATA 2019-11-20 08:53:06.915671


In [33]:
df_compare = df_all4[['trade_date','tot']].merge(df_spy2[['trade_date','spy']],on='trade_date',how='inner')

In [34]:
df_compare['port_pct'] = df_compare.tot / df_compare.iloc[0].tot
df_compare['spy_pct'] = df_compare.spy / df_compare.iloc[0].spy
df_compare['port_pctchg'] = df_compare.tot.pct_change()
df_compare['spy_pctchg'] = df_compare.spy.pct_change()
port_mean = df_compare.port_pctchg.mean() * 256
port_std = df_compare.port_pctchg.std() * 256**.5
spy_mean = df_compare.spy_pctchg.mean() * 256
spy_std = df_compare.spy_pctchg.std() * 256**.5
{'port_mean':port_mean,'port_std':port_std,'spy_mean':spy_mean,'spy_std':spy_std,
 'port_sharpe':port_mean/port_std,'spy_sharpe':spy_mean/spy_std}

{'port_mean': 0.022600766187192037,
 'port_std': 0.15793669061923457,
 'spy_mean': 0.11343976689722696,
 'spy_std': 0.13646469665589747,
 'port_sharpe': 0.14310016310066692,
 'spy_sharpe': 0.8312755582732945}

### Plot portfolio value vs SPY value over whole time period

In [35]:
df_to_plot = df_compare[['trade_date','port_pct','spy_pct']].copy()
t = 'growth of portfolio vs growth of SPY'
iplot(pu.plotly_pandas(df_to_plot,x_column='trade_date',use_secondary_yaxis=True,plot_title=t))

### Do Dividend analysis of portfolio

#### Get dividends from yahoo actions

In [36]:
dict_div = {}
df_div = None
for i in range(len(df_port)):
    symbol = df_port.iloc[i].symbol
    shares = df_port.iloc[i].position
    df_div2 = pu.pdr.DataReader(symbol, 'yahoo-actions').sort_index()
    df_div2['trade_date'] = df_div2.index.year*100*100 + df_div2.index.month*100 + df_div2.index.day
    df_div2.index = range(len(df_div2))
    df_div3 = df_div2.merge(df_all[['trade_date',symbol]],on='trade_date',how='inner')
    df_div3['close'] = (df_div3[symbol] / shares)
    df_div3['dividend'] = df_div3.value
    df_div3['shares'] = shares
    dict_div[symbol] = df_div3.copy()


In [37]:
dict_div2 = {}
for symbol in dict_div.keys():
    dict_div2[symbol] = dict_div[symbol][['trade_date','close','dividend','shares']]

#### Calculate avergage annual dividend rates for each symbol, and then for whole portfolio

In [52]:
syms = []
rates = []
for symbol in dict_div2.keys():
    dft = dict_div2[symbol]
    dft = dft[dft.trade_date<20190000]
    dft['year'] = dft.trade_date.astype(str).str.slice(0,4).astype(int)
    dft2 = dft[['year','dividend','close']].groupby('year',as_index=False).agg({'dividend': 'sum', 'close': 'mean'})
    rate = dft2.apply(lambda r:r.dividend/r.close,axis=1).mean()
    syms.append(symbol)
    rates.append(rate)
avgs = [df_all[c].mean() for c in syms]
df_div_rate = pd.DataFrame({'symbol':syms,'div_rate':rates,'avg_close':avgs})
portfolio_dividend_rate = df_div_rate.apply(lambda r:r.div_rate * r.avg_close,axis=1).sum() / df_div_rate.avg_close.sum()
print(f'portfolio dividend rate = {portfolio_dividend_rate}')

portfolio dividend rate = 0.05339348095371684


In [54]:
dft2

,year,dividend,close
0,2014,0.240,7.547563
1,2017,0.371,7.463741
2,2018,0.604,7.677141


#### Do Dividend analysis of SPY

In [40]:
df_div_spy = pu.pdr.DataReader('SPY', 'yahoo-actions').sort_index()
df_div_spy['trade_date'] = df_div_spy.index.year*100*100 + df_div_spy.index.month*100 + df_div_spy.index.day
df_div_spy.index = range(len(df_div_spy))
df_div_spy = df_div_spy[(df_div_spy.trade_date>20160000) & (df_div_spy.trade_date<20190000)]

In [41]:
df_div_spy['year'] = df_div_spy.trade_date.astype(str).str.slice(0,4).astype(int)
df_div_spy_agg = df_div_spy[['year','value']].groupby('year',as_index=False).agg({'value':sum})
df_spy2 = df_spy[df_spy.trade_date>20160000].copy()
df_spy2['year'] = df_spy2.trade_date.astype(str).str.slice(0,4).astype(int)
df_spy_close_avg = df_spy2[['year','close']].groupby('year',as_index=False).mean()
df_spy_div = df_div_spy_agg.merge(df_spy_close_avg,on='year',how='inner')
df_spy_div['div_rate'] = df_spy_div.value/df_spy_div.close
df_spy_div

,year,value,close,div_rate
0,2016,4.539,196.401009,0.023111
1,2017,4.802,234.186343,0.020505
2,2018,5.101,267.391619,0.019077


### Get Expense Ratios for each portfolio member using web scrapes of Yahoo Finance

In [42]:
sel = sela.SelScrape()

In [43]:
syms = []
expense_ratios = []
for symbol in dict_div2.keys():
    try:
        print(f'getting symbol {symbol}')
        sel.goto(f'https://finance.yahoo.com/quote/{symbol}/profile?p={symbol}')
        xp = "//span[contains(text(),'Annual Report Expense Ratio')]/parent::*/parent::span/following-sibling::span"
        p = sel.findxpath(xpath=xp)['value'] 
        v = float(re.findall('[.0-9]+',p[0].text)[0])
        syms.append(symbol)
        expense_ratios.append(v)
    except Exception as e:
        print(symbol,e)
df_expense_ratios = pd.DataFrame({'symbol':syms,'expense_ratio':expense_ratios})
df_expense_ratios

getting symbol XGPMX
XGPMX list index out of range


,symbol,expense_ratio


In [44]:
aer = df_expense_ratios.expense_ratio.mean()
print(f'average expense ratio = {aer}')

average expense ratio = nan


## END